In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
import datetime

In [11]:
# Requests, getting html
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
r = requests.get(url).text

In [141]:
# Using BeautifulSoup to parse html
soup = BeautifulSoup(r, 'html.parser')
table = soup.find_all('tbody')
rows = table[2].find_all('tr')

In [189]:
# For loop to build table
# 213 countries
# 'Country' and 'GDP_USD_billion.'
count = 0 
df = pd.DataFrame(columns=['Country', 'GDP_USD_billion'])
data_dict = dict()

for row in rows[3:]:
    if count < 214:
        col = row.find_all('td')
        data_dict = {'Country':col[0].contents[2],
                     'GDP_USD_billion':col[2].contents[0]}
        df1 = pd.DataFrame(data_dict, index = [0])
        df = pd.concat([df,df1])
        count+=1
    else:
        break

df

,Country,GDP_USD_billion
0,United States,"26,854,599"
0,China,"19,373,586"
0,Japan,"4,409,738"
0,Germany,"4,308,854"
0,India,"3,736,882"
...,...,...
0,Anguilla,—
0,Kiribati,248
0,Nauru,151
0,Montserrat,—


In [215]:
# Data cleaning and formating
df['GDP_USD_billion'] = df['GDP_USD_billion'].replace('—',np.nan, regex=True)
df['GDP_USD_billion'] = df['GDP_USD_billion'].replace(',','', regex=True).astype(float)
df['GDP_USD_billion'] = round(df['GDP_USD_billion']/1000,2)
df = df.sort_values(by='GDP_USD_billion', ascending=False)
df

,Country,GDP_USD_billion
0,United States,26854.60
0,China,19373.59
0,Japan,4409.74
0,Germany,4308.85
0,India,3736.88
...,...,...
0,Turks and Caicos Islands,NaN
0,Sint Maarten,NaN
0,Cook Islands,NaN
0,Anguilla,NaN


In [195]:
# Saving to CSV
df = df.sort_values(by='GDP_USD_billion', ascending=False)
df.to_csv(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\DataEngCourse\Week_2_PracticeProject\Countries_by_GDP.csv')

file_name = 'Countries_by_GDP.csv'
csv_path = fr'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\DataEngCourse\Week_2_PracticeProject\{file_name}'
df.to_csv(csv_path, index = False)

In [200]:
# Uploading dataframe to database
database = 'World_Economies.db'
table_name = 'World_Economies'

conn = sqlite3.connect(database)
df.to_sql(table_name, conn, index = False, if_exists='replace')
conn.close()

In [214]:
# Query on database
conn = sqlite3.connect(database)
cursor = conn.cursor()
cursor.execute("select * from World_Economies where GDP_USD_billion > 100")
rows = cursor.fetchall()

In [213]:
pd.DataFrame(rows, columns=['Country','GDP_USD_billion'])

,Country,GDP_USD_billion
0,United States,26854599.0
1,China,19373586.0
2,Japan,4409738.0
3,Germany,4308854.0
4,India,3736882.0
...,...,...
64,Kenya,118130.0
65,Angola,117877.0
66,Oman,104902.0
67,Guatemala,102309.0
